In [19]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import os, pickle

SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
creds = None

if os.path.exists('token.pkl'):
    with open('token.pkl', 'rb') as token:
        creds = pickle.load(token)

if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(r"C:\Users\FINE LAPTOP\Desktop\Langchain\langgraph\client_secret_360782607957-dtr7rusrjsi5r558fiv6g60qg9930hmn.apps.googleusercontent.com.json", SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.pkl', 'wb') as token:
        pickle.dump(creds, token)


service = build('drive', 'v3', credentials=creds)
results = service.files().list(
    q="mimeType='application/pdf' or mimeType='application/vnd.google-apps.document'",
    pageSize=10,
    fields="files(id, name, mimeType)"
).execute()

files = results.get('files', [])
for f in files:
    print(f"{f['name']} ({f['id']}) - {f['mimeType']}")


Shahzain -1.pdf (1Pb987kphhul9ZeXtGwzRIzv4GyYfyJWl) - application/pdf
Innovaxel | Take Home Assignment | Backend - Summer Intern (1BoQA7nEsrK6ah5fNRG47BXC6W3ZnG_l0sGz2WnxqUDE) - application/vnd.google-apps.document
Shah cv.pdf (1Cq7MriFdOQA98wbDOvXRPuYnsYx4G0zE) - application/pdf
Shah cv.pdf (1ueJS99efnAtlL5XStBSZJuS3LKmgw3q2) - application/pdf
Shah cv.pdf (1p-xCIVbxye3DITf4V5vbylCjCpGba9RR) - application/pdf
Shah cv.pdf (1p3ghmdyBLXzrh5viqrDkd6C0IBApMWd8) - application/pdf
Shah Zain final1 (2).pdf (14eOI0HkDgotroXhYPfyC6nPzsSXE-zO5) - application/pdf
Shah Zain (1).pdf (1uR_nrad4DyuG9cu9DmmmUYxa1wkgk5YL) - application/pdf
Shah Zain (1).pdf (1WH1A6YxGvwD_ZXg3iztvpRev-s16E0D-) - application/pdf
2D3MF.pdf (10gjCXD-Bkpe5J_U6OYoDl57_HDTo6K4Q) - application/pdf


In [20]:
import io
from googleapiclient.http import MediaIoBaseDownload

file_id = "1WigYzL1C9d-m47oA-1_w5ceSRJ5GSUGU" 
request = service.files().get_media(fileId=file_id)

fh = io.BytesIO()
downloader = MediaIoBaseDownload(fh, request)
done = False
while not done:
    status, done = downloader.next_chunk()

with open("document.txt", "wb") as f:
    f.write(fh.getvalue())


In [21]:
from dotenv import load_dotenv
from langchain_core.documents import Document
from langchain_community.vectorstores import Qdrant
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnableLambda


load_dotenv()

def load_txt_as_documents(txt_file):
    with open(txt_file, 'r', encoding='utf-8') as f:
        raw_text = f.read()
    return raw_text


raw_text = load_txt_as_documents(r"C:\Users\FINE LAPTOP\Desktop\Langchain\langgraph\document.txt")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(raw_text)
documents = [Document(page_content=chunk) for chunk in texts]


embedding_function = OpenAIEmbeddings()


qdrant_client = QdrantClient(host="localhost", port=6333)

qdrant_client.recreate_collection(
    collection_name="rag_txt_collection",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)


db = Qdrant(
    client=qdrant_client,
    collection_name="rag_txt_collection",
    embeddings=embedding_function
)


db.add_documents(documents)

print("Uploaded documents to running Docker Qdrant.")




Uploaded documents to running Docker Qdrant.


In [28]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI


class GraphState(TypedDict):
    question: str
    context: str
    answer: str


llm = ChatOpenAI(model="gpt-3.5-turbo")


def retrieve(state: GraphState):
    query = state["question"]
    retriever = db.as_retriever()
    docs = retriever.invoke(query)
    context = "\n\n".join([doc.page_content for doc in docs])
    return {"question": query, "context": context}


def generate(state: GraphState):
    prompt = f"""Answer the question using this context:\n\n{state['context']}\n\nQuestion: {state['question']}"""
    response = llm.invoke(prompt)
    return {
        "question": state["question"],
        "context": state["context"],
        "answer": response.content
    }


graph = StateGraph(GraphState)
graph.add_node("retrieve", RunnableLambda(retrieve))
graph.add_node("generate", RunnableLambda(generate))
graph.set_entry_point("retrieve")
graph.add_edge("retrieve", "generate")
graph.add_edge("generate", END)
app = graph.compile()


inputs = {"question": "I have an issue setting a different delivery address up"}
result = app.invoke(inputs)

In [ ]:
def pretty_print_answer(question: str, answer: str):
    print("❓ User Question:")
    print("-" * 80)
    print(question.strip())
    print("-" * 80)

    print("📬 AI Response:")
    print("-" * 80)
    
    lines = answer.strip().split("\n")
    for line in lines:
        line = line.strip()
        if line.startswith(("1.", "2.", "3.", "4.", "5.", "6.", "7.", "8.", "9.")):
            print(f"{line}  ") 
        elif line == "":
            print()
        else:
            print(line)

    print("-" * 80)

pretty_print_answer(result["question"], result["answer"])


❓ User Question:
--------------------------------------------------------------------------------
I have an issue setting a different delivery address up
--------------------------------------------------------------------------------
📬 AI Response:
--------------------------------------------------------------------------------
Answer: It seems like you're encountering difficulties setting up a different delivery address, and I'm here to assist you with that. To address this issue, please follow these steps:

1. Log in to your account.  
2. Go to the 'Delivery Addresses' or 'Shipping Settings' section.  
3. Look for the option to 'Add New Address' or 'Edit Address'.  
4. Enter the details of your new delivery address accurately.  
5. Save the changes and ensure that the new address is set up correctly.  

If you need further assistance or encounter any obstacles during this process, please don't hesitate to reach out to me. I'm here to help resolve any issues you may have with setting